In [3]:
import pandas as pd
import numpy as np

# 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [21]:
attendance_df = pd.read_csv('untidy-data/attendance.csv')
attendance_df.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [22]:
attendance_melt = attendance_df.melt(id_vars=['Unnamed: 0'])
attendance_melt.head()

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [23]:
mapping = {'A': 1, 'H': .5, 'T': .1, 'P': 0}

# Replace values in the 'column_name' column using the mapping
attendance_melt['value'] = attendance_melt['value'].map(mapping)

In [24]:
attendance_melt.head()

,Unnamed: 0,variable,value
0,Sally,2018-01-01,0.0
1,Jane,2018-01-01,1.0
2,Billy,2018-01-01,1.0
3,John,2018-01-01,0.0
4,Sally,2018-01-02,0.1


In [26]:
1-attendance_melt.groupby('Unnamed: 0').mean()

/var/folders/qh/kjmdb2zx3n3b2ng_n4q3cyn00000gn/T/ipykernel_45846/1960734550.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  1-attendance_melt.groupby('Unnamed: 0').mean()


,value
Unnamed: 0,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


You should end up with something like this:

    `name
    Billy    0.5250
    Jane     0.6875
    John     0.9125
    Sally    0.7625
    Name: grade, dtype: float64`


# 2. Coffee Levels

* a. Read the coffee_levels table.

In [27]:
cof_df = pd.read_csv('untidy-data/coffee_levels.csv')
cof_df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


* b. Transform the data so that each carafe is in it's own column.

In [29]:
cof_df.pivot_table(values='coffee_amount', index='hour', columns='coffee_carafe')

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


* c. Is this the best shape for the data?

In [ ]:
Yes

# 3. Cake Recipes

* a. Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [31]:
cake_df = pd.read_csv('untidy-data/cake_recipes.csv')
cake_df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


* b. Tidy the data as necessary.

* c. Which recipe, on average, is the best?

* d. Which oven temperature, on average, produces the best results?

* e. Which combination of recipe, rack position, and temperature gives the best result?

Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.